# Chain
In the world of artificial intelligence, large language models (LLMs) have emerged as powerful tools for generating text, translating languages, writing creative content, and answering questions in an informative way. However, when it comes to complex tasks that require multiple steps, managing the flow of data and ensuring seamless integration between different components can become challenging. This is where chains in LangChain come into play.

Chains provide a structured and organized approach for connecting different components, whether they are LLMs, validation modules, parsers, or other tools, to create a pipeline that efficiently processes data and handles complex tasks. By defining a chain, you essentially establish a sequence in which each component is executed, ensuring that the output of one component seamlessly flows into the next

With chains, you can harness the power of multiple LLMs and other components to tackle complex tasks efficiently and effectively. Whether you're developing chatbots, generating creative content, or performing advanced data analysis, chains provide a powerful tool for managing the flow of data and ensuring seamless integration between different components.

* [langchain website](https://python.langchain.com/docs/modules/chains/)
* [why using LCEL](https://python.langchain.com/docs/expression_language/why)
* [Interface](https://python.langchain.com/docs/expression_language/interface)
* [LLMChain api](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html?highlight=llmchain#langchain.chains.llm.LLMChain)

## Initialization section

In [ ]:
!pip install langchain
!pip install openai==v0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


## Import section

In [ ]:
import os


from langchain.chat_models import ChatOpenAI

from langchain.prompts import ChatPromptTemplate

from langchain.schema import StrOutputParser
from langchain.output_parsers.list import CommaSeparatedListOutputParser


from langchain.chains import LLMChain


## preparation

In [ ]:
openai_api_key = ...
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
llm = ChatOpenAI(temperature=0)

## create a chain
We are in the process of constructing a chain designed to receive a food name as input and subsequently provide a list of required ingredients.

In [ ]:
output_parser = CommaSeparatedListOutputParser()
output_parser_format =  output_parser.get_format_instructions()

In [ ]:
promt_food_ingredients = """
You are a helpful AI that will provide a list of ingredients needed to cook a dish.

format of output: {format_output}
"""

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", promt_food_ingredients),
        ("user", "{food_name}"),
    ],
)
chat_template = chat_template.partial(format_output=output_parser_format)

You can use | as pipe symbol.

In [ ]:
food_name = input("Which specific dish would you like to know the ingredients for?")
chain = chat_template | llm | output_parser

output = chain.invoke({"food_name":food_name})
print(f"""
type:{type(output)}


contain:{output}
""")

Which specific dish would you like to know the ingredients for?pizza

type:<class 'list'>


contain:['dough', 'tomato sauce', 'cheese', 'toppings (e.g. pepperoni', 'mushrooms', 'onions', 'etc.)']



for this task you can use LLMChain too.

In [ ]:
food_name = input("Which specific dish would you like to know the ingredients for?")

llm_chain = LLMChain(llm=llm, prompt=chat_template,output_parser=output_parser)
output = llm_chain({"food_name":food_name})
print(f"""
type:{type(output)}


contain:{output}
""")

Which specific dish would you like to know the ingredients for?pizza

type:<class 'dict'>


contain:{'food_name': 'pizza', 'text': ['dough', 'tomato sauce', 'cheese', 'toppings (e.g. pepperoni', 'mushrooms', 'onions', 'etc.)']}



## a little bit complex chain
We are developing an app that takes the name of a food as input. Initially, it generates a recipe for the specified food. Subsequently, the app passes this recipe to another Language Model Module (LLM) to assess the level of difficulty involved in cooking the food.

In [ ]:
promt_food_recipe = """
As a helpful AI, your task is to provide a recipe for the specific food that the user has asked about..

"""
promt_recipe_difficulty= """
As a helpful AI, your role is to determine the level of difficulty in cooking the recipe provided by the user.

"""

In [ ]:
recipe_template = ChatPromptTemplate.from_messages(
    [
        ("system", promt_food_recipe),
        ("user", "{food_name}"),
    ],
)

difficulty_template = ChatPromptTemplate.from_messages(
    [
        ("system", promt_recipe_difficulty),
        ("user", "{recipe}"),
    ],
)


In [ ]:
food_name = input("Which specific dish would you like to know the ingredients for?")
chain = recipe_template | llm | {"recipe":StrOutputParser()}|difficulty_template | llm |StrOutputParser()

output = chain.invoke({"food_name":food_name})
print(f"""
type:{type(output)}


contain:{output}
""")

Which specific dish would you like to know the ingredients for?pizza

type:<class 'str'>


contain:The difficulty level of this homemade pizza recipe is considered to be moderate. While the steps are relatively straightforward, there are a few techniques involved that may require some practice, such as kneading the dough and rolling it out to the desired thickness. Additionally, the rising time for the dough can take up to 2 hours, which requires some patience. However, with careful attention to the instructions, even beginner cooks should be able to successfully make this homemade pizza.



## Task
Create a program that takes a country name as input and returns a list of its cities.